# Clase 1.4 - Polimorfismo y Metodos Magicos en Python

**Unidad:** 1 - POO Avanzada con Python  
**Conexion al Proyecto TaskFlow:** Implementaremos metodos magicos para que nuestros modelos sean mas intuitivos y faciles de usar.  

## 📋 Objetivos de Aprendizaje

Al finalizar esta clase, seras capaz de:
- [ ] Implementar `__str__` y `__repr__` para representacion de objetos
- [ ] Sobrecargar operadores (`__add__`, `__eq__`, `__lt__`, etc.)
- [ ] Comprender el concepto de duck typing en Python
- [ ] Crear objetos comparables y ordenables
- [ ] Aplicar metodos magicos al sistema TaskFlow

## 💻 Conexion con TaskFlow

En el sistema **TaskFlow**, usaremos metodos magicos para:
- Representar tareas de forma legible (`__str__`)
- Depurar con informacion completa (`__repr__`)
- Comparar tareas por prioridad (`__lt__`, `__gt__`)
- Verificar si dos tareas son iguales (`__eq__`)
- Obtener longitud de listas de tareas (`__len__`)

In [ ]:
# Importaciones
from dataclasses import dataclass, field
from datetime import datetime
from typing import Optional, List
from enum import Enum

---

## 1. 📘 Metodos `__str__` y `__repr__`

**📐 Diferencia clave:**

| Metodo | Proposito | Usado en |
|--------|-----------|----------|
| **`__str__`** | Representacion legible para usuarios | `print()`, `str()` |
| **`__repr__`** | Representacion completa para debugging | `repr()`, consola interactiva |

**📋 Regla general:**
- `__str__`: Corto, amigable, para usuarios finales
- `__repr__`: Completo, sin ambigüedades, para desarrolladores
- Si no defines `__str__`, Python usa `__repr__` como fallback

**💡 Recuerda:** `__repr__` deberia idealmente ser un string que podria usarse para recrear el objeto (ej: `eval(repr(obj))`)

In [ ]:
# Ejemplo 1: __str__ y __repr__ basicos

class Prioridad(Enum):
    """Prioridades de tareas."""
    BAJA = 1
    MEDIA = 2
    ALTA = 3
    URGENTE = 4

@dataclass
class Tarea:
    """Tarea en TaskFlow."""
    id: int
    titulo: str
    estado: str = "pendiente"
    prioridad: Prioridad = Prioridad.MEDIA
    
    def __str__(self) -> str:
        """
        Representacion legible para usuarios.
        Usado por print() y str().
        """
        emoji_estado = self._emoji_estado()
        emoji_prioridad = self._emoji_prioridad()
        return f"{emoji_estado} {self.titulo} {emoji_prioridad}"
    
    def __repr__(self) -> str:
        """
        Representacion completa para debugging.
        Usado por repr() y en consola interactiva.
        """
        return (f"Tarea(id={self.id}, titulo='{self.titulo}', "
                f"estado='{self.estado}', prioridad={self.prioridad.name})")
    
    def _emoji_estado(self) -> str:
        """Retorna emoji segun estado."""
        emojis = {
            "pendiente": "\찴",
            "en_progreso": "\",
            "completada": "\u2705",
            "cancelada": "\u274c"
        }
        return emojis.get(self.estado, "\u2753")
    
    def _emoji_prioridad(self) -> str:
        """Retorna emoji segun prioridad."""
        emojis = {
            Prioridad.BAJA: "\u2b07\ufe0f",
            Prioridad.MEDIA: "\u23fa",
            Prioridad.ALTA: "\u26a0\ufe0f",
            Prioridad.URGENTE: "\"
        }
        return emojis.get(self.prioridad, "")

# Crear tareas
t1 = Tarea(id=1, titulo="Implementar auth", estado="completada", prioridad=Prioridad.ALTA)
t2 = Tarea(id=2, titulo="Crear UI", estado="en_progreso", prioridad=Prioridad.MEDIA)
t3 = Tarea(id=3, titulo="Fix bug", estado="pendiente", prioridad=Prioridad.URGENTE)

# Usar __str__ (para usuarios)
print("=== REPRESENTACION LEGIBLE (__str__) ===")
for tarea in [t1, t2, t3]:
    print(f"  {tarea}")  # Usa __str__

# Usar __repr__ (para debugging)
print("\n=== REPRESENTACION COMPLETA (__repr__) ===")
print(f"  {repr(t1)}")
print(f"  {repr(t2)}")

# En consola interactiva, __repr__ se usa por defecto
print("\n=== EN LISTA (usa __repr__) ===")
print(f"  {[t1, t2, t3]}")

**💡 Nota:** Observa como `print()` usa `__str__` pero las listas usan `__repr__` para mostrar sus elementos.

---

## 2. ⚙️ Sobrecarga de Operadores de Comparacion

**📐 Definicion:** La **sobrecarga de operadores** permite definir como se comportan los operadores (`==`, `<`, `>`, etc.) con objetos de tu clase.

**📋 Metodos de comparacion:**

| Operador | Metodo | Significado |
|----------|--------|-------------|
| `==` | `__eq__` | Igualdad |
| `!=` | `__ne__` | Desigualdad |
| `<` | `__lt__` | Menor que |
| `<=` | `__le__` | Menor o igual |
| `>` | `__gt__` | Mayor que |
| `>=` | `__ge__` | Mayor o igual |

In [ ]:
# Ejemplo 2: Comparacion de tareas por prioridad

@dataclass
class TareaComparable:
    """Tarea que se puede comparar por prioridad."""
    id: int
    titulo: str
    prioridad: Prioridad = Prioridad.MEDIA
    
    def __eq__(self, other: object) -> bool:
        """
        Verifica si dos tareas son iguales.
        Dos tareas son iguales si tienen el mismo ID.
        """
        if not isinstance(other, TareaComparable):
            return NotImplemented
        return self.id == other.id
    
    def __lt__(self, other: 'TareaComparable') -> bool:
        """
        Menor que: compara por prioridad.
        Menor prioridad = menor valor numerico.
        """
        if not isinstance(other, TareaComparable):
            return NotImplemented
        return self.prioridad.value < other.prioridad.value
    
    def __le__(self, other: 'TareaComparable') -> bool:
        """Menor o igual que."""
        if not isinstance(other, TareaComparable):
            return NotImplemented
        return self.prioridad.value <= other.prioridad.value
    
    def __gt__(self, other: 'TareaComparable') -> bool:
        """Mayor que."""
        if not isinstance(other, TareaComparable):
            return NotImplemented
        return self.prioridad.value > other.prioridad.value
    
    def __ge__(self, other: 'TareaComparable') -> bool:
        """Mayor o igual que."""
        if not isinstance(other, TareaComparable):
            return NotImplemented
        return self.prioridad.value >= other.prioridad.value
    
    def __str__(self) -> str:
        return f"{self.titulo} (Prioridad: {self.prioridad.name})"

# Crear tareas con diferentes prioridades
t_baja = TareaComparable(1, "Documentar codigo", Prioridad.BAJA)
t_media = TareaComparable(2, "Crear mockups", Prioridad.MEDIA)
t_alta = TareaComparable(3, "Fix critical bug", Prioridad.ALTA)
t_urgente = TareaComparable(4, "Security fix", Prioridad.URGENTE)
t_alta2 = TareaComparable(3, "Otra tarea alta", Prioridad.ALTA)  # Mismo ID que t_alta

# Probar igualdad (por ID)
print("=== IGUALDAD (por ID) ===")
print(f"t_alta == t_alta2: {t_alta == t_alta2}")  # True (mismo ID)
print(f"t_alta == t_urgente: {t_alta == t_urgente}")  # False (diferente ID)

# Probar comparaciones (por prioridad)
print("\n=== COMPARACION DE PRIORIDAD ===")
print(f"t_baja < t_alta: {t_baja < t_alta}")  # True
print(f"t_alta > t_media: {t_alta > t_media}")  # True
print(f"t_urgente >= t_alta: {t_urgente >= t_alta}")  # True
print(f"t_media <= t_media: {t_media <= t_media}")  # True

# Ordenar tareas (usa __lt__)
print("\n=== TAREAS ORDENADAS POR PRIORIDAD ===")
tareas = [t_urgente, t_baja, t_alta, t_media]
tareas_ordenadas = sorted(tareas)
for tarea in tareas_ordenadas:
    print(f"  - {tarea}")

**💡 Nota:** Al implementar `__lt__`, `sorted()` puede ordenar objetos de tu clase automaticamente.

---

## 3. ⚖️ Atajo: `@total_ordering`

**📐 Definicion:** `@total_ordering` es un decorador que genera automaticamente todos los operadores de comparacion si implementas `__eq__` y uno de `__lt__`, `__le__`, `__gt__`, o `__ge__`.

**📋 Ventaja:** Menos codigo, menos propenso a errores.

In [ ]:
# Ejemplo 3: Usando @total_ordering

from functools import total_ordering

@dataclass
@total_ordering
class TareaSimplificada:
    """
    Tarea comparable con @total_ordering.
    Solo necesita __eq__ y __lt__ (o __gt__, __le__, __ge__).
    """
    id: int
    titulo: str
    prioridad: Prioridad = Prioridad.MEDIA
    
    def __eq__(self, other: object) -> bool:
        """Igualdad por ID."""
        if not isinstance(other, TareaSimplificada):
            return NotImplemented
        return self.id == other.id
    
    def __lt__(self, other: 'TareaSimplificada') -> bool:
        """Menor que por prioridad."""
        if not isinstance(other, TareaSimplificada):
            return NotImplemented
        return self.prioridad.value < other.prioridad.value

# Crear tareas
t1 = TareaSimplificada(1, "Tarea baja", Prioridad.BAJA)
t2 = TareaSimplificada(2, "Tarea urgente", Prioridad.URGENTE)

# Todos los operadores funcionan
print("=== OPERADORES GENERADOS AUTOMATICAMENTE ===")
print(f"t1 < t2: {t1 < t2}")
print(f"t1 > t2: {t1 > t2}")
print(f"t1 <= t2: {t1 <= t2}")
print(f"t1 >= t2: {t1 >= t2}")
print(f"t1 == t2: {t1 == t2}")
print(f"t1 != t2: {t1 != t2}")

---

## 4. 👑 Otros Metodos Magicos Utiles

### `__len__`: Longitud personalizada

Permite usar `len()` con objetos de tu clase.

### `__bool__`: Valor de verdad

Define si un objeto es `True` o `False` en contextos booleanos.

### `__hash__`: Hash personalizado

Permite usar objetos como claves en diccionarios (si son inmutables).

In [ ]:
# Ejemplo 4: Otros metodos magicos

@dataclass
class Proyecto:
    """Proyecto con metodos magicos adicionales."""
    id: int
    nombre: str
    tareas: List[TareaSimplificada] = field(init=False, default_factory=list)
    
    def agregar_tarea(self, tarea: TareaSimplificada) -> None:
        self.tareas.append(tarea)
    
    def __len__(self) -> int:
        """
        Retorna la cantidad de tareas.
        Permite usar len(proyecto).
        """
        return len(self.tareas)
    
    def __bool__(self) -> bool:
        """
        Un proyecto es 'truthy' si tiene tareas.
        """
        return len(self.tareas) > 0
    
    def __contains__(self, tarea_id: int) -> bool:
        """
        Permite usar 'in' para verificar si una tarea existe.
        """
        return any(t.id == tarea_id for t in self.tareas)
    
    def __iter__(self):
        """
        Permite iterar sobre el proyecto (itera sus tareas).
        """
        return iter(self.tareas)
    
    def __str__(self) -> str:
        return f"Proyecto: {self.nombre} ({len(self)} tareas)"

# Crear proyecto
p = Proyecto(id=1, nombre="TaskFlow")

# Probar metodos magicos
print("=== PROYECTO SIN TAREAS ===")
print(f"Proyecto: {p}")
print(f"len(proyecto): {len(p)}")
print(f"bool(proyecto): {bool(p)}")  # False (no tiene tareas)
print(f"if proyecto: 'Si' else 'No': {'Si' if p else 'No'}")

# Agregar tareas
p.agregar_tarea(TareaSimplificada(1, "Tarea 1", Prioridad.ALTA))
p.agregar_tarea(TareaSimplificada(2, "Tarea 2", Prioridad.MEDIA))
p.agregar_tarea(TareaSimplificada(3, "Tarea 3", Prioridad.BAJA))

print("\n=== PROYECTO CON TAREAS ===")
print(f"Proyecto: {p}")
print(f"len(proyecto): {len(p)}")
print(f"bool(proyecto): {bool(p)}")  # True (tiene tareas)
print(f"1 in proyecto: {1 in p}")  # True (tarea con ID 1 existe)
print(f"5 in proyecto: {5 in p}")  # False (tarea con ID 5 no existe)

# Iterar sobre el proyecto
print("\n=== ITERAR SOBRE PROYECTO ===")
for tarea in p:
    print(f"  - {tarea.titulo}")

**💡 Nota:** Al implementar `__iter__`, puedes usar tu clase en bucles `for` directamente.

---

## 5. 🥛 Duck Typing en Python

**📐 Definicion:** **Duck typing** significa que "si camina como un pato y grazna como un pato, entonces es un pato". En Python, el tipo de un objeto se determina por su comportamiento (metodos y atributos), no por su clase explicita.

**📋 Ventaja:** Mayor flexibilidad y polimorfismo sin necesidad de herencia explicita.

**⚠️ Requisito:** El objeto debe tener los metodos/atributos necesarios para funcionar.

In [ ]:
# Ejemplo 5: Duck typing en accion

@dataclass
class Usuario:
    """Usuario que puede tener tareas asignadas."""
    username: str
    tareas: List['TareaSimplificada'] = field(init=False, default_factory=list)
    
    def agregar_tarea(self, tarea: 'TareaSimplificada') -> None:
        self.tareas.append(tarea)
    
    def __len__(self) -> int:
        """Cantidad de tareas asignadas."""
        return len(self.tareas)


def reportar_carga(objeto) -> str:
    """
    Funcion que funciona con cualquier objeto que tenga __len__.
    No verifica el tipo, solo el comportamiento (duck typing).
    """
    cantidad = len(objeto)  # Funciona si objeto tiene __len__
    return f"Tiene {cantidad} elementos"


# Crear objetos de diferentes tipos
proyecto = Proyecto(id=1, nombre="TaskFlow")
proyecto.agregar_tarea(TareaSimplificada(1, "T1", Prioridad.ALTA))
proyecto.agregar_tarea(TareaSimplificada(2, "T2", Prioridad.MEDIA))

usuario = Usuario("jdoe")
usuario.agregar_tarea(TareaSimplificada(3, "T3", Prioridad.BAJA))

lista = [1, 2, 3, 4, 5]

# Todos funcionan con reportar_carga (duck typing)
print("=== DUCK TYPING ===")
print(f"Proyecto: {reportar_carga(proyecto)}")
print(f"Usuario: {reportar_carga(usuario)}")
print(f"Lista: {reportar_carga(lista)}")

# Intentar con objeto sin __len__
try:
    reportar_carga("hola")  # str tiene __len__, pero length es de caracteres
    print(f"String: {reportar_carga('hola')}")
except TypeError as e:
    print(f"Error: {e}")

**💡 Explicacion:** `reportar_carga()` no verifica si el objeto es un `Proyecto`, `Usuario` o `lista`. Solo verifica si tiene `__len__()`.

---

## 6. 📋 Resumen de Metodos Magicos Comunes

| Categoria | Metodo | Se usa con |
|-----------|--------|-----------|
| **Representacion** | `__str__` | `print()`, `str()` |
| **Representacion** | `__repr__` | `repr()`, consola |
| **Comparacion** | `__eq__` | `==` |
| **Comparacion** | `__lt__` | `<` |
| **Comparacion** | `__gt__` | `>` |
| **Comparacion** | `__le__` | `<=` |
| **Comparacion** | `__ge__` | `>=` |
| **Aritmeticos** | `__add__` | `+` |
| **Aritmeticos** | `__sub__` | `-` |
| **Contenedores** | `__len__` | `len()` |
| **Contenedores** | `__contains__` | `in` |
| **Contenedores** | `__getitem__` | `obj[key]` |
| **Iteracion** | `__iter__` | `for` loops |
| **Booleanos** | `__bool__` | `bool()`, `if` |
| **Hash** | `__hash__` | `dict` keys, `set` |

---

## 📝 Ejercicio Practico: Sistema de Prioridad de Tareas

Completa la clase `TareaPrioritaria` que:

1. Tiene `id`, `titulo`, `prioridad` (1-5)
2. `__str__` retorna formato: `[ID] Titulo (Prioridad X)`
3. `__repr__` retorna formato: `TareaPrioritaria(id=X, titulo='...', prioridad=X)`
4. `__eq__` compara por `id`
5. `__lt__` compara por `prioridad` (menor numero = menor prioridad)
6. `__bool__` retorna `True` si prioridad >= 3

In [ ]:
# Tu solucion aqui
from dataclasses import dataclass

@dataclass
class TareaPrioritaria:
    """Tarea con prioridad y metodos magicos."""
    id: int
    titulo: str
    prioridad: int = 3  # 1-5, donde 5 es mas alta
    
    # TODO: Completar __str__
    def __str__(self) -> str:
        pass
    
    # TODO: Completar __repr__
    def __repr__(self) -> str:
        pass
    
    # TODO: Completar __eq__ (compara por id)
    def __eq__(self, other: object) -> bool:
        pass
    
    # TODO: Completar __lt__ (compara por prioridad)
    def __lt__(self, other: 'TareaPrioritaria') -> bool:
        pass
    
    # TODO: Completar __bool__ (True si prioridad >= 3)
    def __bool__(self) -> bool:
        pass

### ✅ Validacion Automatica

In [ ]:
# Tests automaticos

def test_tarea_prioritaria():
    """Valida la clase TareaPrioritaria."""
    
    t1 = TareaPrioritaria(1, "Tarea baja", 1)
    t2 = TareaPrioritaria(2, "Tarea media", 3)
    t3 = TareaPrioritaria(3, "Tarea alta", 5)
    t4 = TareaPrioritaria(1, "Mismo ID", 5)  # Mismo ID que t1
    
    # Test 1: __str__
    str_t1 = str(t1)
    assert "1" in str_t1 and "Tarea baja" in str_t1 and "Prioridad 1" in str_t1
    print("✅ Test 1: __str__ funciona")
    
    # Test 2: __repr__
    repr_t1 = repr(t1)
    assert "TareaPrioritaria" in repr_t1 and "id=1" in repr_t1
    print("✅ Test 2: __repr__ funciona")
    
    # Test 3: __eq__ (por ID)
    assert t1 == t4  # Mismo ID
    assert t1 != t2  # Diferente ID
    print("✅ Test 3: __eq__ funciona (compara por ID)")
    
    # Test 4: __lt__ (por prioridad)
    assert t1 < t2  # 1 < 3
    assert t2 < t3  # 3 < 5
    assert t1 < t3  # 1 < 5
    print("✅ Test 4: __lt__ funciona (compara por prioridad)")
    
    # Test 5: __bool__
    assert not bool(t1)  # Prioridad 1 < 3
    assert bool(t2)     # Prioridad 3 >= 3
    assert bool(t3)     # Prioridad 5 >= 3
    print("✅ Test 5: __bool__ funciona (True si prioridad >= 3)")
    
    # Test 6: Ordenamiento
    tareas = [t3, t1, t2]
    ordenadas = sorted(tareas)
    assert ordenadas[0] == t1  # Prioridad 1
    assert ordenadas[1] == t2  # Prioridad 3
    assert ordenadas[2] == t3  # Prioridad 5
    print("✅ Test 6: sorted() funciona correctamente")
    
    print("\n🚀 iTodos los tests pasaron!")

# Ejecutar tests
try:
    test_tarea_prioritaria()
except (AssertionError, TypeError) as e:
    print(f"\n❌ {e}")
    print("\nRevisa tu implementacion.")

### 📋 Solucion del Ejercicio

In [ ]:
# Solucion completa

@dataclass
class TareaPrioritaria:
    """Tarea con prioridad y metodos magicos."""
    id: int
    titulo: str
    prioridad: int = 3  # 1-5, donde 5 es mas alta
    
    def __str__(self) -> str:
        """Representacion legible."""
        return f"[{self.id}] {self.titulo} (Prioridad {self.prioridad})"
    
    def __repr__(self) -> str:
        """Representacion para debugging."""
        return f"TareaPrioritaria(id={self.id}, titulo='{self.titulo}', prioridad={self.prioridad})"
    
    def __eq__(self, other: object) -> bool:
        """Igualdad por ID."""
        if not isinstance(other, TareaPrioritaria):
            return NotImplemented
        return self.id == other.id
    
    def __lt__(self, other: 'TareaPrioritaria') -> bool:
        """Menor que por prioridad."""
        if not isinstance(other, TareaPrioritaria):
            return NotImplemented
        return self.prioridad < other.prioridad
    
    def __bool__(self) -> bool:
        """True si la prioridad es media o alta (>= 3)."""
        return self.prioridad >= 3

# Ejecutar tests de nuevo
test_tarea_prioritaria()

---

## 📈 Diagrama: Metodos Magicos y Contexto de Uso

```mermaid
graph TD
    A[print/tarea] --> B[__str__]
    C[repr/tarea] --> D[__repr__]
    E[tarea1 == tarea2] --> F[__eq__]
    G[tarea1 < tarea2] --> H[__lt__]
    I[len/proyecto] --> J[__len__]
    K[for tarea in proyecto] --> L[__iter__]
    M[tarea_id in proyecto] --> N[__contains__]
    O[if tarea] --> P[__bool__]
    
    style B fill:#e1f5ff
    style D fill:#c8e6c9
    style F fill:#fff9c4
    style H fill:#ffccbc
```

---

## 📝 Resumen de la Clase

### 📋 Conceptos Clave

| Concepto | Proposito | Ejemplo |
|----------|-----------|---------|
| **`__str__`** | Representacion amigable | `print(tarea)` |
| **`__repr__`** | Representacion completa | `repr(tarea)` |
| **`__eq__`** | Definir igualdad | `tarea1 == tarea2` |
| **`__lt__`** | Habilitar `<` y `sorted()` | `tarea1 < tarea2` |
| **`__len__`** | Habilitar `len()` | `len(proyecto)` |
| **`__bool__`** | Definir valor de verdad | `if tarea:` |
| **Duck typing** | Polimorfismo por comportamiento | Cualquier objeto con `__len__` |

### 📋 Buenas Practicas

1. **Siempre implementa `__repr__`** para debugging facil
2. **Implementa `__str__`** para representacion amigable
3. **Usa `@total_ordering`** para evitar repetir codigo de comparacion
4. **Verifica tipos** con `isinstance()` antes de comparar
5. **Retorna `NotImplemented`** si el tipo es incorrecto

### 🤝 Conexion con TaskFlow

Hemos implementado:
- Representaciones legibles de tareas (`__str__`)
- Comparacion de tareas por prioridad (`__lt__`, `__gt__`)
- Igualdad de tareas por ID (`__eq__`)
- Proyectos medibles por cantidad de tareas (`__len__`)
- Iteracion sobre proyectos (`__iter__`)

**📈 Proxima clase:** Interfaces y ABCs

Aprenderemos a:
- Crear clases abstractas con `ABC`
- Definir metodos abstractos con `@abstractmethod`
- Usar Protocolos para type checking estructural
- Disenar interfaces de Repository para TaskFlow